In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import json
from matplotlib import container
from helper_functions import *

In [ ]:
df = pd.read_csv('../data/gpt_high_conf.csv', index_col=0)
repl = ['tiiuae_', '_responses.json','mistralai_','meta-llama_','_responses.json']
df['model'] = df['model'].replace(repl,  '', regex=True).str.lower()
# Drop empty responses
df = df[df['answer'].notna()]
subset = df[df['label']!='unrelated'].reset_index()

In [ ]:
subset = subset[subset['statement']!='statement']
pd.unique(subset['statement'])

In [ ]:
pd.unique(subset['model'])

In [ ]:
models = [m for m in pd.unique(subset['model']) if 'gpt_4' in m]

In [ ]:

groups = ['opposite', 'reformulation']

gpt_bias_dict = {m:{} for m in models}
gpt_bias_dict

In [ ]:
for m in models:
    model_subset = subset[subset['model'] == m]
    for g in groups:
        gpt_bias_dict[m][g] = (compute_bias(model_subset, 'statement', g))


In [ ]:
df = pd.read_csv('../data/formulations_high_conf.csv', index_col=0)
repl = ['tiiuae_', '_responses.json','mistralai_','meta-llama_','_responses.json']
df['model'] = df['model'].replace(repl,  '', regex=True).str.lower()
# Drop empty responses
df = df[df['answer'].notna()]
df = df[df['prefix'].notna()]
subset = df[df['label']!='unrelated']

In [ ]:
subset['statement_type'] = subset['prefix'].apply(lambda x: 'opposite' if 'opposite' in x else 'reformulation')

In [ ]:
models = list(pd.unique(subset['model']))
groups = ['opposite', 'reformulation']

llama_bias_dict = {m:{} for m in models}


In [ ]:
subset.loc[subset['statement_type']== 'opposite', 'label'] = subset.loc[subset['statement_type']== 'opposite']['label'].replace({'agree':'disagree', 'disagree':'agree'})

In [ ]:
for m in models:
    model_subset = subset[subset['model'] == m]
    for g in groups:
        llama_bias_dict[m][g] = compute_bias(model_subset, 'statement_type', g)


In [ ]:
llama_bias = pd.DataFrame(llama_bias_dict).T.reset_index(names='model')
llama_bias['model'] = llama_bias['model'].str.replace('responses_', '').str.replace('.json', '')
llama_bias

In [ ]:
gpt_bias = pd.DataFrame(gpt_bias_dict).T.reset_index(names='model')
llama_bias['model'] = llama_bias['model'].str.replace('responses_', '').str.replace('.json', '')
gpt_bias['model'] = gpt_bias['model'].str.replace('gpt_4', 'gpt-4')

In [ ]:
df = pd.merge(left=gpt_bias, right=llama_bias, on='model', suffixes=['_gpt', '_llama'])
compare = pd.DataFrame({'compare' : df['model'], 'Opposite': pd.NA, 'Reformulation': pd.NA})
compare.Opposite = df['opposite_llama'].abs() -df['opposite_gpt'].abs()

compare.Reformulation = df['reformulation_llama'].abs() -df['reformulation_gpt'].abs()
compare.to_csv('./data/formulation_compare.csv')